In [ ]:
import numpy as np
from importlib import reload
import sys
sys.path.insert(1, './utils')

import torch
print(torch.cuda.get_device_name(torch.cuda.current_device()))
print('CUDA Version ' + torch.version.cuda)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import wandb
%env WANDB_SILENT=True
%env "WANDB_NOTEBOOK_NAME" "gnn-case118-random-sweep"

## Load data from file

In [ ]:
import dataparamsgnn
reload(dataparamsgnn)

In [ ]:
dataParamsRandom = dataparamsgnn.dataParams(device, 'IEEE118', ['random'], 'Train')
nodeScale = torch.std(torch.abs(dataParamsRandom.nodeDataTrain[:,:,0] - dataParamsRandom.nodeDataTrain[:,:,1]))

In [ ]:
scalingList = np.arange(10,21)/10
dataParamsTest = dataparamsgnn.dataParams(device, 'IEEE118', scalingList, 'Test', dataParamsRandom.numOutputLabels)

## Define model training function

In [ ]:
import nnmodel
reload(nnmodel)
import nntrainer
reload(nntrainer)
import livemetric
reload(livemetric)
liveMetricCalculator = livemetric.liveMetricCalculator(device, dataParamsRandom, dataParamsTest)

In [ ]:
def defineModelAndTrain():
    wandb.init(project='gnn-power-cascade-case118-sweep')

    np.random.seed(0)
    torch.manual_seed(0)

    numNodeFeatures = 1
    numHiddenFeatures   = wandb.config.numHiddenFeatures
    hiddenLayerDepth    = wandb.config.hiddenLayerDepth
    numAvgLayers        = wandb.config.numAvgLayers
    attentionLayerDepth = wandb.config.attentionLayerDepth
    attentionLayerWidth = wandb.config.attentionLayerWidth

    M_lines = dataParamsRandom.linkSet.shape[0]

    pfModel = nnmodel.GCN(numNodeFeatures, numHiddenFeatures, numAvgLayers, \
                          attentionLayerDepth, attentionLayerWidth, hiddenLayerDepth, dataParamsRandom.numOutputLabels, \
                          dataParamsRandom.node2edgeAdjMatrix, dataParamsRandom.edge2edgeAdjMatrix, nodeScale, 0, False, device).to(device)

    learningRate = wandb.config.learningRate
    batchSize    = wandb.config.batchSize
    weightDecay  = wandb.config.weightDecay
    gammaVal     = wandb.config.gammaVal*1e-5+0.9999
    maxEpochs    = wandb.config.maxEpochs

    criterion = torch.nn.CrossEntropyLoss(reduction='none')

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, pfModel.parameters()), lr=learningRate, betas=(0.9, 0.999), weight_decay = weightDecay)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gammaVal)

    nnTrainer = nntrainer.nnTrainer(device, criterion, scheduler)

    pfModel.train()

    try:
        for epoch in range(maxEpochs):         
            nodeFeatures, edgeFeatures, activeEdges = dataParamsRandom.getTrainBatch(batchSize)
            currLoss = nnTrainer.batchStep(nodeFeatures, edgeFeatures, activeEdges, pfModel, optimizer)
            
            liveMetricCalculator.batchRoutine(currLoss, epoch, pfModel, scalingList, 128)
            if(np.isnan(currLoss) or currLoss > 1e4): break
    except KeyboardInterrupt:
        print('Interrupted.') 
    liveMetricCalculator.postRunRoutine(pfModel, 128)

## Define sweep configurations and run the sweep

In [ ]:
sweep_configuration = {
    'method': 'bayes',
    'metric': 
    {
        'goal': 'maximize', 
        'name': 'min-accuracy'
        },
    'parameters': 
    {
        'numHiddenFeatures'   : {'values': [150]},
        'hiddenLayerDepth'    : {'values': [2]},
        'numAvgLayers'        : {'values': [15]},
        'attentionLayerDepth' : {'values': [30]},
        'attentionLayerWidth' : {'values': [2048]},
        'learningRate' : {'min': 2.75e-4, 'max': 3e-4},
        'batchSize'    : {'values': [128]},
        'weightDecay'  : {'values': [0]},
        'gammaVal'     : {'min': 1.0, 'max': 9.0},
        'maxEpochs'    : {'values': [5000]},
     }
}

In [ ]:
sweep_id = wandb.sweep(
    sweep=sweep_configuration, 
    project='gnn-power-cascade-case118-sweep'
    )
wandb.agent(sweep_id, function=defineModelAndTrain, count=20)
wandb.finish()